# Thing I wish we had time to present

[Pierre-Antoine Champin](http://champin.net/)

http://github.com/pchampin/rust-w3c-2022

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/2.0/fr/"><img alt="Contrat Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/2.0/fr/88x31.png" /></a>

# Unit tests

In [2]:
fn fact(n: usize) -> usize {
    if n <= 1 { 1 } else { n*fact(n-1) }
}

#[cfg(test)]
mod test {
    #[test]
    fn fact0() {
        assert_eq!(fact(0), 1)
    }

    #[test]
    fn fact1() {
        assert_eq!(fact(1), 1)
    }

    #[test]
    fn fact10() {
        assert_eq!(fact(10), 3628800)
    }    
}


# Smart pointers

`Box<T>`, `Rc<T>`, `Arc<T>`, `RefCell<T>`, `Mutex<T>`, `RwLock<T>`...

# Trait objects and dynamic dispatch

## Interlude

In C++

```cpp
class Animal {
  private:
      const char* name;
  public:
      Animal(const char* name) { this->name = name; }
      const char* get_name() { return this->name; }
      const char* cry() { return "noise"; }
};    

class Dog: public Animal {
  public:
      Chien(const char* name): Animal(name) {}
      const char* cry() { return "bark"; }
};
```

Quizz: what would this print?
```cpp
void make_speak(Animal *a) {
  cout << a->get_name() << " makes a " << a->cry() << ".\n";
}

// (...)

  make_speak(new Dog("Rex"))
```

... unless I marked the `cry` method in `Animal` as `abstract`.

What about in Java, then?

... unless I marked the `cry` method in `Animal` as `final`.

(in which case I would not be allowed to override it in `Dog`)

## In Rust

In [3]:
trait Animal: std::fmt::Debug {
    fn cry(&self) -> &str { "makes a noise" }
}

#[derive(Debug)]
struct Dog;
impl Animal for Dog {
    fn cry(&self) -> &str { "barks" }
}

#[derive(Debug)]
struct Platypus;
impl Animal for Platypus {
    // inherits the default implementation of cry()
}

In [4]:
fn all_cry1<T>(zoo: &[T])
where T: Animal,
{
    for a in zoo {
        println!("{:?} {}", a, a.cry());
    }
}

all_cry1(&[Platypus, Platypus]);
all_cry1(&[Dog, Dog, Dog]);
// can not mix Platypuses and Dogs in the same call

Platypus makes a noise
Platypus makes a noise
Dog barks
Dog barks
Dog barks


In [5]:
fn all_cry2(zoo: &[&dyn Animal]) {
    for a in zoo {
        println!("{:?} {}", a, a.cry());
    }
}
let (a1, a2, a3) = (Dog, Platypus, Dog);
all_cry2(&[&a1, &a2, &a3]);

Dog barks
Platypus makes a noise
Dog barks


# Iterators

In [6]:
let a = [1, 2, 3];
{
    let v: Vec<_> = a
            .iter()
            .rev()
            .cycle()
            .skip(1)
            .take(6)
            .collect();
    v
}


[2, 1, 3, 2, 1, 3]

# Closures

In [7]:
let a = [1, 2, 1, 3, 1, 4, 1, 5];
{
    let v: Vec<_> = a
        .iter()
        .map(|x| 2*x)
        //.take_while(|x| *x<7)
        //.filter(|x| *x>2)
        .collect();
    v
}

[2, 4, 2, 6, 2, 8, 2, 10]

# Multithreading

In [8]:
use std::thread;
use std::time::Duration;

static VALUES: [i32; 3] = [2, 4, 6];

thread::spawn(|| {
    for v in &VALUES {
        thread::sleep(Duration::from_millis(1000));
        println!("A {}", v);
    }
});

thread::sleep(Duration::from_millis(1500));
for v in &VALUES {
    thread::sleep(Duration::from_millis(1000));
    println!("B {}", v);
}


A 2
A 4
B 2
A 6
B 4
B 6


()

## Multithreading with `rayon`

In [9]:
// mono-thread task

let mut v: Vec<i32> = (0..10).collect();
v.iter_mut().for_each(|i| {
    print!("{} ", i);
    *i += 1;
});
println!("-> {:?}", v);

0 1 2 3 4 5 6 7 8 9 -> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [22]:
:dep rayon = "*"
use rayon::prelude::*;

// same task, parallelized with rayon
let mut v: Vec<i32> = (0..10).collect();
v.par_iter_mut().for_each(|i| {
    print!("{} ", i);
    *i += 10;
});
println!("-> {:?}", v);

0 1 7 6 8 9 5 2 3 4 -> [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


# `async` concurrency

# The `serde` crate (Serializer / Deserializer)

In [11]:
:dep serde = {version="1.0", features=["derive"]}
:dep serde_cbor = "0.11.2"
:dep serde_json = "1.0"
:dep serde_yaml = "0.8"
:dep serde_qs = "0.9"

In [12]:
use serde::{Deserialize, Serialize};
use std::error::Error;
use std::fs::File;

// Types annotated with `Serialize` can be stored as CBOR.
// To be able to load them again add `Deserialize`.
#[derive(Debug, Serialize, Deserialize)]
struct Mascot {
    name: String,
    species: String,
    year_of_birth: Option<i64>,
}

let ferris = Mascot {
    name: "Ferris".to_owned(),
    species: "crab".to_owned(),
    year_of_birth: Some(2015),
};

## JSON

In [13]:
println!("{}", serde_json::to_string_pretty(&ferris)?);

{
  "name": "Ferris",
  "species": "crab",


In [14]:
let m: Mascot = serde_json::from_str(r#"
{
  "year_of_birth": 1996,
  "name": "Tux",
  "species": "penguin"
}
"#)?;
m

  "year_of_birth": 2015
}


Mascot { name: "Tux", species: "penguin", year_of_birth: Some(1996) }

## URL query string

In [15]:
serde_qs::to_string(&ferris)?

"name=Ferris&species=crab&year_of_birth=2015"

In [16]:
let m: Mascot = serde_qs::from_str("name=Tux&species=nice+penguin")?;
m

Mascot { name: "Tux", species: "nice penguin", year_of_birth: None }

## Yaml

In [17]:
println!("{}", serde_yaml::to_string(&ferris)?);

---
name: Ferris
species: crab
year_of_birth: 2015



## CBOR

In [18]:
let mut v: Vec<u8> = serde_cbor::to_vec(&ferris)?;
v

[163, 100, 110, 97, 109, 101, 102, 70, 101, 114, 114, 105, 115, 103, 115, 112, 101, 99, 105, 101, 115, 100, 99, 114, 97, 98, 109, 121, 101, 97, 114, 95, 111, 102, 95, 98, 105, 114, 116, 104, 25, 7, 223]

In [19]:
let hex: String = v.iter().map(|b| format!("{:02x}", b)).collect();
println!("Inspect CBOR at https://cbor.me/?bytes={}", &hex);

Inspect CBOR at https://cbor.me/?bytes=a3646e616d6566466572726973677370656369657364637261626d796561725f6f665f62697274681907df


In [20]:
let m: Mascot = serde_cbor::from_slice(&v)?;
m

Mascot { name: "Ferris", species: "crab", year_of_birth: Some(2015) }

In [21]:
let mut v2 = v.clone();
v2[7] += 32;        // lowercase first letter of name
v2[0] = 0b10100010; // change map length to 2
v2.truncate(26);    // truncate third item
let m2: Mascot = serde_cbor::from_slice(&v2)?;
m2

Mascot { name: "ferris", species: "crab", year_of_birth: None }

# `unsafe` code

# Web Assembly

# Sockets